In [3]:
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


In [4]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)



# https://stats.stackexchange.com/questions/108007/correlations-with-unordered-categorical-variables

## read traces from hadoop

In [5]:
df = spark.read.json("/user/sprenner/training_all_april.json")

In [6]:
df.describe().toPandas()

summary  account           accountHash         clientState  \
0  count   3231296  3231296               3231296              
1  mean    None     -6.862163155897912E7  None                 
2  stddev  None     1.214873416428219E9   None                 
3  min     aad      -2144719344           CannotAuthenticate   
4  max     zzheng   2143592175            SystemError          

        clientStateHash  \
0  3231296                
1  -9.350691361360884E8   
2  2.5054777314132002E8   
3  -1922357674            
4  1269532611             

                                                                                 dataset  \
0  3231296                                                                                 
1  None                                                                                    
2  None                                                                                    
3  BBS_M1000_nominal_100418                                                                
4  valid3.110401.PowhegPythia_P2012_ttbar_nonallhad.simul.log.e2928_s3213_tid12503332_00   

             datasetHash datasetScope      datasetScopeHash eventType  \
0  3231296                3231296      3231296               3231296    
1  -4.5230852327388145E7  None         -4.507730727803677E8  None       
2  1.245301801667085E9    None         1.0869769945929074E9  None       
3  -2147475923            cond09_mc    -2145215727           download   
4  2147475364             valid3       2131974379            download   

  eventTypeHash eventVersion      eventVersionHash  \
0  3231296       3231296      3231296                
1  5.937808E7    None         -1.963768093741567E9   
2  0.0           None         3.8513566029732066E8   
3  59378080      1.13.2       -2044360925            
4  59378080      api          1261702107             

                           filename          filenameHash  \
0  3231296                           3231296                
1  None                              -1742584.3853425994    
2  None                              1.2397161559696255E9   
3  AOD.05371635._000001.pool.root.1  -2147481484            
4  wjets_v1.15_small.root            2147482617             

        filesize_double               hits                     hostname  \
0  3231296               3231296            3231296                       
1  3.482938720038774E8   5.085354916417438  None                          
2  1.0850033994592447E9  35.12657136873029  None                          
3  5.0                   1                  130-199-21-196.dhcp.bnl.gov   
4  3.0587790406E10       1348               xenia2.nevis.columbia.edu     

           hostnameHash protocol         protocolHash  \
0  3231296               3231296  3231296               
1  9.92930343842449E7    None     4.964437819779924E8   
2  1.3049200474702454E9  None     9.39691729069515E8    
3  -2142788649           davs     -1607963002           
4  2143857082            srm      2128034789            

                 remoteSite        remoteSiteHash      scope  \
0  3231296                   3231296               3231296     
1  None                      3.4601683712934375E7  None        
2  None                      1.2936656207502983E9  None        
3  AGLT2_CALIBDISK           -2135244854           cond09_mc   
4  ZA-WITS-CORE_SCRATCHDISK  2141740830            valid3      

              scopeHash                           traceId  \
0  3231296               3231296                            
1  -4.625174831601209E8  Infinity                           
2  1.084347690419062E9   NaN                                
3  -2145215727           00000a90a2c841548def8b19b974a7bf   
4  2131974379            fffffa77425f474b9fa9d590bdc6e23c   

            traceIdHash                    traceIp            traceIpHash  \
0  3231296               3231296                    3231296                 
1  -620846.8997167081    None                       -2.196431865064500

## ANOVA function

In [7]:
# lol: diffSq wird doch eigentlich immer null wenn ich für jeden entry die anzahl der downloads dieses monats habe,
# da ist der wert gleich dem mittelwert und folglich ist ist diffSq 0 (value - average). die summe davon ist auch 0,
# und im endeffekt ist somit msw 0 und verursacht eine dividebyzero exception


# Implementation of ANOVA function: calculates the degrees of freedom, F-value, eta squared and omega squared values.
# Expects that 'categoryData' with two columns, the first being the categorical independent variable and the second being the scale dependent variable
# Berechnung: http://www.methodenberatung.uzh.ch/de/datenanalyse/unterschiede/zentral/evarianz.html
# https://statistics.laerd.com/statistical-guides/one-way-anova-statistical-guide.php

def getAnovaStats(categoryData) : 
    #try:
        print(str(categoryData.columns))
        cat_val = categoryData.toDF("cat","value")
        cat_val.createOrReplaceTempView("df")
        newdf = spark.sql("select A.cat, A.value, cast((A.value * A.value) as double) as valueSq, ((A.value - B.avg) * (A.value - B.avg)) as diffSq from df A join (select cat, avg(value) as avg from df group by cat) B where A.cat = B.cat")
        grouped = newdf.groupBy("cat")
        sums = grouped.sum("value")
        counts = grouped.count()
        numCats = counts.count() 
        sumsq = grouped.sum("valueSq") 
        avgs = grouped.avg("value")
    
        totN = counts.selectExpr("sum(count) as total").rdd.map(lambda x: x.total).collect()[0]
        totSum = sums.selectExpr("sum(`sum(value)`) as totSum").rdd.map(lambda x: x.totSum).collect()[0]
        totSumSq = sumsq.selectExpr("sum(`sum(valueSq)`) as totSumSq").rdd.map(lambda x: x.totSumSq).collect()[0]

        print("totN:" + str(totN) + " totSum: " + str(totSum) + " totSumSq: " + str(totSumSq) + " sums: " + str(sums) + " counts: " + str(counts) + " numCats: " + str(numCats) + " sumsq: " + str(sumsq) + " avgs: " + str(avgs))
       
        totMean = totSum / totN

        #dft = totN - 1
        # degrees of freedom between groups
        dfb = numCats - 1
        # degrees of freedom within groups
        dfw = totN - numCats
    
        joined = counts.selectExpr("cat as category", "count").join(sums, col("category") == sums.cat, 'inner')\
            .drop(sums.cat)\
            .join(sumsq, col("category") == sumsq.cat, 'inner')\
            .drop(sumsq.cat)\
            .join(avgs, col("category") == avgs.cat, 'inner')\
            .drop(avgs.cat)
        finaldf = joined.withColumn("totMean", lit(totMean))

        # sum of squares between groups
        ssb_tmp = finaldf.rdd.map(lambda x: (x[0], ((x[4] - x[5])*(x[4] - x[5]))*x[1]))
        ssb = ssb_tmp.toDF().selectExpr("sum(_2) as total").rdd.map(lambda x: x.total).collect()[0]

        # sum of squares within groups
        ssw_tmp = grouped.sum("diffSq")
        ssw = ssw_tmp.selectExpr("sum(`sum(diffSq)`) as total").rdd.map(lambda x: x.total).collect()[0]
    
        # sum of squares total
        sst = ssb + ssw

        # mean squares between groups
        print("msb: " + str(ssb) + "/" + str(dfb))
        if dfb == 0:
            return ('only 1 unique value in categorical column, no variance', 0, 0, 0, 0)
        msb = ssb / dfb
        # mean squares within groups
        print("msw: " + str(ssw) + "/" + str(dfw))
        if ssw == 0:
            return ('ssw is zero', 0, 0, 0, 0)
        msw = ssw / dfw
        # statistic F
        print("F: " + str(msb) + "/" + str(msw))
        F = msb / msw

        # partial eta square => percent of variance of the dependent variable that is predicted by the independent variable
        # when you only have one independent variable, partial eta2 is the same as eta2
        # http://www.statisticshowto.com/eta-squared/
        etaSq = ssb / sst
    
        # Omega squared is widely viewed as a lesser biased alternative to eta-squared, especially when sample sizes are small.
        # ω2 can have values between ± 1.
        # Zero indicates no effect.
        # If the observed F is less than one, ω2 will be negative.
        # http://www.statisticshowto.com/omega-squared/
        omegaSq = (ssb - ((numCats - 1) * msw))/(sst + msw)
        return (dfb, dfw, F, etaSq, omegaSq)
    #except Exception as ex:
        #template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        #message = template.format(type(ex).__name__, ex.args)
        #print message
        #return ('error', message, 0, 0, 0)

In [8]:
#df_cleaned.select('account','hits').limit(10).toPandas()

In [9]:
getAnovaStats(df.select('account','hits'))

['account', 'hits']
totN:3231296 totSum: 16432287 totSumSq: 4070581413.0 sums: DataFrame[cat: string, sum(value): bigint] counts: DataFrame[cat: string, count: bigint] numCats: 894 sumsq: DataFrame[cat: string, sum(valueSq): double] avgs: DataFrame[cat: string, avg(value): double]
msb: 3259377952.45/893
msw: 727662990.552/3230402
F: 3649919.31965/225.254624828


(893, 3230402, 16203.526664305402, 0.8174929726193126, 0.8174424748904086)

In [10]:
#categorical = ['account','appid','clientState','dataset','datasetName','datasetScope','dataset_scope','duid','eventType','eventVersion','filename','guid','hostname','ip','localSite','name','pq','protocol','remoteSite','scope','stateReason','suspicious','taskid','traceId','traceIp','url','usr','usrdn','uuid','version','project','run_number','stream_name','prod_step','datatype','dataset_version']
#result_list = []
#for elem in categorical:
#    stats = getAnovaStats(df_cleaned.select(elem, 'hits'))
#    result_list.append({'column': elem, 'dfb': stats[0], 'dfw': stats[1], 'F': stats[2], 'etaSq': stats[3], 'omegaSq': stats[4]})
#    
#df_anova = pd.DataFrame(result_list)

In [11]:
getAnovaStats(df.select('clientState','hits'))

['clientState', 'hits']
totN:3231296 totSum: 16432287 totSumSq: 4070581413.0 sums: DataFrame[cat: string, sum(value): bigint] counts: DataFrame[cat: string, count: bigint] numCats: 13 sumsq: DataFrame[cat: string, sum(valueSq): double] avgs: DataFrame[cat: string, avg(value): double]
msb: 472718939.002/12
msw: 3514322003.99/3231283
F: 39393244.9168/1087.59338133


(12, 3231283, 36220.56330353955, 0.11856385368520106, 0.11856054795886627)

In [12]:
getAnovaStats(df.select('dataset','hits'))

['dataset', 'hits']
totN:3231296 totSum: 16432287 totSumSq: 4070581413.0 sums: DataFrame[cat: string, sum(value): bigint] counts: DataFrame[cat: string, count: bigint] numCats: 157316 sumsq: DataFrame[cat: string, sum(valueSq): double] avgs: DataFrame[cat: string, avg(value): double]
msb: 3540874542.96/157315
msw: 446166400.045/3073980
F: 22508.1813111/145.142909207


(157315, 3073980, 155.07599671279527, 0.8880958569467402, 0.8823689820141875)

In [13]:
cat_val = df.select('dataset','hits').toDF("cat","value")
cat_val.createOrReplaceTempView("df_")
newdf = spark.sql("select A.cat, A.value, cast((A.value * A.value) as double) as valueSq, ((A.value - B.avg) * (A.value - B.avg)) as diffSq from df_ A join (select cat, avg(value) as avg from df_ group by cat) B where A.cat = B.cat")
newdf.limit(50).toPandas()
#grouped = newdf.groupBy("cat")

cat  \
0   data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
1   data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
2   data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
3   data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
4   data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
5   data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
6   data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
7   data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
8   data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
9   data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
10  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
11  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
12  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
13  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
14  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
15  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
16  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
17  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
18  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
19  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
20  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
21  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
22  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
23  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
24  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
25  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
26  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
27  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
28  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
29  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
30  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
31  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
32  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
33  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
34  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
35  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
36  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
37  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
38  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
39  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
40  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
41  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
42  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
43  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
44  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
45  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
46  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
47  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
48  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
49  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   

    value  valueSq  diffSq  
0   1      1.

In [14]:
newdf2 = spark.sql("select cat, avg(value) as avg from df_ group by cat")
newdf2.createOrReplaceTempView("avg_cats")
newdf2.limit(50).toPandas()

cat  \
0   data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372                                                       
1   data16_13TeV.00300487.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372                                                       
2   mc16_13TeV.364141.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV1000_E_CMS.deriv.NTUP_PILEUP.e5307_e5984_s3126_r9781_r9778_p3384     
3   mc16_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.deriv.NTUP_PILEUP.e3601_e5984_s3244_r10236_r9778_p3384                 
4   group.perf-idtracking.APv01.00309674.physics_Main.AOD.r9264_p3083_UpsilonmumuValidationUserSel                                
5   group.phys-higgs.data17_13TeV.00334890.CAOD_HIGG4D1.f867_m1860_p3372.Hmumu-r31-01_CxAOD.root                                  
6   group.phys-susy.data15_13TeV.00280673.physics_Main.SusyNt.r9264_p3083_p3372_n0300_nt                                          
7   group.phys-top.410472.PhPy8EG.DAOD_TOPQ1.e6348_e5984_s3126_r9781_r9778_p3409.270318_noSys_output_at21222.root                 
8   user.aqureshi.data16_13TeV.00307732.physics_Main.deriv.DAOD_SUSY1.r9264_p3083_p3372__fds_o.root                               
9   user.areiss.MJ301.MC16a.mc16_13TeV.361025.Pythia8EvtGen_A14NNPDF23LO_jetjet_JZ5W_minitrees.root                               
10  user.bcarlson.data17_13TeV.00339070.physics_EnhancedBias.CPUApril8_EXT1                                                       
11  user.cgutscho.20180329.mc16_13TeV.364193.Sh221_Wtaunu_MAXHTPTV280_500_CVetoBVeto.deriv.DAOD_STDM4.Skim2_v1_outputXAODStream   
12  user.chhayes.PowhegPy8EG_NNPDF30_AZNLOCTEQ6L1_VBF125_a60a60_bbtautau_lh.APR25v1_EXT0                                          
13  user.fawilk.AllYear.physics_Main.DAOD_TOPQ1.grp16_v01_p3229.BCF.20180414r1_output.root                                        
14  user.flaruffa.mc15_13TeV.410000.ATLAS_2017_I1614149.v01.log                                                                   
15  user.fthiele.HNL.364182.Sherpa_221_NNPDF30NNLO_Wenu_MAXHTPTV500_1000.e5340_s2726_r7772_r7676_p2949.v26.1_hist                 
16  user.fthiele.HNL.364197.Sherpa_221_NNPDF30NNLO_Wtaunu_MAXHTPTV1000_E_CMS.e5340_s2726_r7772_r7676_p2949.v26.1_hist             
17  user.ftresold.toyMC_fullRun2_coverage_v2_nToys10000_BRBs1.18_BRBd0.64_outputXYZ.root.tgz                                      
18  user.hod.DAT15d20042018v1.data15_13TeV.periodG.physics_Main.log                                                               
19  user.kkrizka.mc16c.361032.Pythia8EvtGen_A14NNPDF23LO_jetjet_JZ12W.fatjet.20180414-01_tree.root                                
20  user.kuno.data16_13TeV.00307394.physics_Main.deriv.DAOD_SUSY1.r9264_p3083_p3372_v129_o.root                                   
21  user.kuno.mc16_13TeV.364120.Sherpa_221_Zee_HTPTV140_280_CVBV.DAOD_SUSY1.e5299_s3126_r9364_r9315_p3401_v129_o.root             
22  user.lcorpe.mc15_13TeV.364125.Sherpa_221_NNPDF30NNLO_Zee_MAXHTPTV280_500_BFilter.e5299.RIVET.LQunfold.BUGFIX.Apr18_v1_EXT0    
23  user.lrossini.data17_13TeV.00329835.physics_Main_R21.DAOD_SUSY6.p3402_s1_tree.root                                            
24  user.mneumann.mc15_13TeV.301057.AZNLOCTEQ6L1_DYtt_45M50-2.xAOD.MTF.0009.6.4.2.1.O.1.0x10e-1.Dp1.Hl.Jt.nO13_EXT0               
25  user.morgens.periodB.grp16_v01_p3372.ML.EF.v1.0_hist                                                                          
26  user.mtoscani.ZJet_2015Data_23Ap.1524509912.0.periodJ.grp15_v01_p3372_hist                                                    
27  user.pgadow.mc16_13TeV.410013.PowhegPythiaEvtGen_P2012_Wt_inclusive_top.QCD200.r9364_r9315_p3371_XAMPP                        
28  user.sangelid.periodC.QmisID_data17v5r6_output.root                                                                           
29  user.shuzhou.Mc16_ttbar1.output2_cutflow.root.182162281                                                                       
30  user.shuzhou.Mc16_ttbar_zzllll.output2_hist.182854954      

In [15]:
newdf3 = spark.sql("select * from df_ join avg_cats where df_.cat = avg_cats.cat")
newdf3.limit(50).toPandas()


cat  \
0   data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
1   data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
2   data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
3   data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
4   data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
5   data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
6   data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
7   data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
8   data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
9   data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
10  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
11  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
12  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
13  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
14  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
15  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
16  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
17  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
18  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
19  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
20  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
21  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
22  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
23  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
24  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
25  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
26  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
27  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
28  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
29  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
30  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
31  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
32  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
33  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
34  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
35  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
36  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
37  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
38  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
39  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
40  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
41  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
42  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
43  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
44  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
45  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
46  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
47  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
48  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   
49  data15_13TeV.00280862.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372   

    value  \
0   1       
1   1       
2  

In [16]:
#df_cleaned.select('dataset').distinct().rdd.map(lambda r: r[0]).collect()

In [17]:
getAnovaStats(df.select('datasetScope','hits'))

['datasetScope', 'hits']
totN:3231296 totSum: 16432287 totSumSq: 4070581413.0 sums: DataFrame[cat: string, sum(value): bigint] counts: DataFrame[cat: string, count: bigint] numCats: 605 sumsq: DataFrame[cat: string, sum(valueSq): double] avgs: DataFrame[cat: string, avg(value): double]
msb: 3318283433.85/604
msw: 668757509.155/3230691
F: 5493846.74478/207.001384272


(604, 3230691, 26540.14495652729, 0.8322672080082898, 0.8322358059956558)

In [18]:
getAnovaStats(df.select('eventType','hits'))

['eventType', 'hits']
totN:3231296 totSum: 16432287 totSumSq: 4070581413.0 sums: DataFrame[cat: string, sum(value): bigint] counts: DataFrame[cat: string, count: bigint] numCats: 1 sumsq: DataFrame[cat: string, sum(valueSq): double] avgs: DataFrame[cat: string, avg(value): double]
msb: 23541.4834323/0


('only 1 unique value in categorical column, no variance', 0, 0, 0, 0)

In [19]:
getAnovaStats(df.select('eventVersion','hits'))

['eventVersion', 'hits']
totN:3231296 totSum: 16432287 totSumSq: 4070581413.0 sums: DataFrame[cat: string, sum(value): bigint] counts: DataFrame[cat: string, count: bigint] numCats: 7 sumsq: DataFrame[cat: string, sum(valueSq): double] avgs: DataFrame[cat: string, avg(value): double]
msb: 3131650103.21/6
msw: 855390839.773/3231289
F: 521941683.868/264.721242753


(6, 3231289, 1971665.2824712885, 0.7854572220336035, 0.7854567715103771)

In [20]:
getAnovaStats(df.select('filename','hits'))

['filename', 'hits']
totN:3231296 totSum: 16432287 totSumSq: 4070581413.0 sums: DataFrame[cat: string, sum(value): bigint] counts: DataFrame[cat: string, count: bigint] numCats: 2180217 sumsq: DataFrame[cat: string, sum(valueSq): double] avgs: DataFrame[cat: string, avg(value): double]
msb: 3987040943.0/2180216
msw: 0.0/1051079
F: 1828.73666783/0.0


ZeroDivisionError: float division by zero

In [21]:
getAnovaStats(df.select('hostname','hits'))

['hostname', 'hits']
totN:3231296 totSum: 16432287 totSumSq: 4070581413.0 sums: DataFrame[cat: string, sum(value): bigint] counts: DataFrame[cat: string, count: bigint] numCats: 1450 sumsq: DataFrame[cat: string, sum(valueSq): double] avgs: DataFrame[cat: string, avg(value): double]
msb: 2000432804.07/1449
msw: 1986608138.93/3229846
F: 1380560.94139/615.078285135


(1449, 3229846, 2244.528826252942, 0.5017336999216118, 0.5015100862402199)

In [22]:
getAnovaStats(df.select('protocol','hits'))

['protocol', 'hits']
totN:3231296 totSum: 16432287 totSumSq: 4070581413.0 sums: DataFrame[cat: string, sum(value): bigint] counts: DataFrame[cat: string, count: bigint] numCats: 6 sumsq: DataFrame[cat: string, sum(valueSq): double] avgs: DataFrame[cat: string, avg(value): double]
msb: 36608260.4817/5
msw: 3950432682.49/3231290
F: 7321652.09634/1222.55590878


(5, 3231290, 5988.8075823197205, 0.009181812026828322, 0.009180276049888973)

In [23]:
getAnovaStats(df.select('remoteSite','hits'))

['remoteSite', 'hits']
totN:3231296 totSum: 16432287 totSumSq: 4070581413.0 sums: DataFrame[cat: string, sum(value): bigint] counts: DataFrame[cat: string, count: bigint] numCats: 258 sumsq: DataFrame[cat: string, sum(valueSq): double] avgs: DataFrame[cat: string, avg(value): double]
msb: 402965021.292/257
msw: 3584075921.71/3231038
F: 1567957.28129/1109.26455266


(257, 3231038, 1413.5106702259015, 0.10106869406488643, 0.10099716406891056)

In [24]:
getAnovaStats(df.select('scope','hits'))

['scope', 'hits']
totN:3231296 totSum: 16432287 totSumSq: 4070581413.0 sums: DataFrame[cat: string, sum(value): bigint] counts: DataFrame[cat: string, count: bigint] numCats: 600 sumsq: DataFrame[cat: string, sum(valueSq): double] avgs: DataFrame[cat: string, avg(value): double]
msb: 571362611.821/599
msw: 3415678331.18/3230696
F: 953860.787681/1057.25773368


(599, 3230696, 902.2027054447461, 0.14330492713504672, 0.14314605023014385)

In [25]:
getAnovaStats(df.select('traceId','hits'))

['traceId', 'hits']
totN:3231296 totSum: 16432287 totSumSq: 4070581413.0 sums: DataFrame[cat: string, sum(value): bigint] counts: DataFrame[cat: string, count: bigint] numCats: 3229803 sumsq: DataFrame[cat: string, sum(valueSq): double] avgs: DataFrame[cat: string, avg(value): double]
msb: 3987040943.0/3229802
msw: 0.0/1493
F: 1234.45367332/0.0


ZeroDivisionError: float division by zero

In [26]:
getAnovaStats(df.select('traceIp','hits'))

['traceIp', 'hits']
totN:3231296 totSum: 16432287 totSumSq: 4070581413.0 sums: DataFrame[cat: string, sum(value): bigint] counts: DataFrame[cat: string, count: bigint] numCats: 1340 sumsq: DataFrame[cat: string, sum(valueSq): double] avgs: DataFrame[cat: string, avg(value): double]
msb: 3248960794.3/1339
msw: 738080148.698/3229956
F: 2426408.3602/228.510898817


(1339, 3229956, 10618.34850224006, 0.8148802183749537, 0.8148034290243159)

In [27]:
getAnovaStats(df.select('uuid','hits'))

['uuid', 'hits']
totN:3231296 totSum: 16432287 totSumSq: 4070581413.0 sums: DataFrame[cat: string, sum(value): bigint] counts: DataFrame[cat: string, count: bigint] numCats: 117389 sumsq: DataFrame[cat: string, sum(valueSq): double] avgs: DataFrame[cat: string, avg(value): double]
msb: 3621040306.18/117388
msw: 366000636.824/3113907
F: 30846.7671838/117.537433464


(117388, 3113907, 262.4420686654945, 0.9082024383353653, 0.9047418291322702)